Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора.

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
df = pd.read_table('AUCs.txt')

/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
df.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

In [4]:
classifier_df = df.drop(columns='Unnamed: 0')

In [5]:
data = []
for i, column in enumerate(classifier_df.columns):
    for j, next_column in enumerate(classifier_df.columns[i + 1 : len(classifier_df.columns)]):
        classifier_AUCs = classifier_df[column]
        next_classifier_AUCs = classifier_df[next_column]
        data.append([column, next_column, stats.wilcoxon(classifier_AUCs, next_classifier_AUCs).pvalue])
p_val_df = pd.DataFrame(data)
p_val_df.columns = ['classifier_A', 'classifier_b', 'p_value']
p_val_df

,classifier_A,classifier_b,p_value
0,C4.5,C4.5+m,0.010757
1,C4.5,C4.5+cf,0.861262
2,C4.5,C4.5+m+cf,0.015906
3,C4.5+m,C4.5+cf,0.046333
4,C4.5+m,C4.5+m+cf,0.327826
5,C4.5+cf,C4.5+m+cf,0.022909


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [6]:
reject, p_corrected, a1, a2 = multipletests(p_val_df.p_value, alpha = 0.05, method = 'holm')

In [7]:
p_corrected

array([0.0645428 , 0.86126233, 0.07953222, 0.13899819, 0.65565135,
       0.0916364 ])

In [8]:
reject

array([False, False, False, False, False, False])

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [9]:
reject, p_corrected, a1, a2 = multipletests(p_val_df.p_value, alpha = 0.05, method = 'fdr_bh')

In [10]:
p_corrected

array([0.0458182 , 0.86126233, 0.0458182 , 0.06949909, 0.39339081,
       0.0458182 ])

In [11]:
reject

array([ True, False,  True, False, False,  True])